In [1]:
import os, sys, re
import pandas as pd
from com.ibm.watson.utils import pubmed_utils as pu
from com.ibm.watson import DESKTOP, DOCS_DIR
from collections import Counter
import nltk
import datetime, random

In [2]:
base_dir = os.path.join(DESKTOP, "Levels_Of_Evidence")

In [3]:
xls = pd.read_excel(os.path.join(base_dir, "Level_of_evidence_20170330_v29.9.2.xlsx"), 
                    dtype={'pubmed_ID' : 'str', 'gene' : 'str'}, engine='xlrd')

In [4]:
genes = set(sum( [x.strip().split(";") for x in xls["gene"]], [] ))
genes_str = " OR ".join( genes )

In [5]:
print genes_str

CDKN2B OR PDGFRA OR ROS1 OR PDGFRB OR nan OR CDKN1A OR ATM OR EGFR OR TNFSF11 OR RET OR STK11 OR AKT1 OR MAP2K1 OR CDKN2A OR ARAF OR PTPN11 OR ABL1 OR RICTOR OR RAF1 OR FLT3 OR FLT1 OR CHIC2 OR MET OR PIK3CA OR NRAS OR NTRK1 OR NTRK2 OR NTRK3 OR TSC1 OR JAK2 OR NF2 OR NF1 OR KRAS OR CHEK2 OR HRAS OR KDR OR CHEK1 OR BRCA1 OR BRCA2 OR PIK3R1 OR IDH2 OR IDH1 OR RAD50 OR MTOR OR TSC2 OR APC OR MDM2 OR EP300 OR FGFR2 OR FGFR3 OR DNMT3A OR FGFR1 OR PTCH1 OR TP53 OR ALK OR ESR1 OR DDR2 OR ARID1A OR ERBB2 OR NOTCH1 OR KIT OR PTEN OR EZH2 OR CDK4 OR BRAF OR GNA11 OR ATR OR FBXW7 OR PALB2 OR CREBBP


In [5]:
genes_re = re.compile("|".join(['\s?' + x + '\s+' for x in genes]), re.IGNORECASE)

In [6]:
search_term = '((((Clinical Study[Publication Type] OR Clinical Trial[Publication Type] OR Journal Article[Publication Type])) AND (CDKN2B[Title/Abstract] OR PDGFRA[Title/Abstract] OR ROS1[Title/Abstract] OR PDGFRB[Title/Abstract] OR CDKN1A[Title/Abstract] OR ATM[Title/Abstract] OR EGFR[Title/Abstract] OR TNFSF11[Title/Abstract] OR RET[Title/Abstract] OR STK11[Title/Abstract] OR AKT1[Title/Abstract] OR MAP2K1[Title/Abstract] OR CDKN2A[Title/Abstract] OR ARAF[Title/Abstract] OR PTPN11[Title/Abstract] OR ABL1[Title/Abstract] OR RICTOR[Title/Abstract] OR RAF1[Title/Abstract] OR FLT3[Title/Abstract] OR FLT1[Title/Abstract] OR CHIC2[Title/Abstract] OR MET[Title/Abstract] OR PIK3CA[Title/Abstract] OR NRAS[Title/Abstract] OR NTRK1[Title/Abstract] OR NTRK2[Title/Abstract] OR NTRK3[Title/Abstract] OR TSC1[Title/Abstract] OR JAK2[Title/Abstract] OR NF2[Title/Abstract] OR NF1[Title/Abstract] OR KRAS[Title/Abstract] OR CHEK2[Title/Abstract] OR HRAS[Title/Abstract] OR KDR[Title/Abstract] OR CHEK1[Title/Abstract] OR BRCA1[Title/Abstract] OR BRCA2[Title/Abstract] OR PIK3R1[Title/Abstract] OR IDH2[Title/Abstract] OR IDH1[Title/Abstract] OR RAD50[Title/Abstract] OR MTOR[Title/Abstract] OR TSC2[Title/Abstract] OR APC[Title/Abstract] OR MDM2[Title/Abstract] OR EP300[Title/Abstract] OR FGFR2[Title/Abstract] OR FGFR3[Title/Abstract] OR DNMT3A[Title/Abstract] OR FGFR1[Title/Abstract] OR PTCH1[Title/Abstract] OR TP53[Title/Abstract] OR ALK[Title/Abstract] OR ESR1[Title/Abstract] OR DDR2[Title/Abstract] OR ARID1A[Title/Abstract] OR ERBB2[Title/Abstract] OR NOTCH1[Title/Abstract] OR KIT[Title/Abstract] OR PTEN[Title/Abstract] OR EZH2[Title/Abstract] OR CDK4[Title/Abstract] OR BRAF[Title/Abstract] OR GNA11[Title/Abstract] OR ATR[Title/Abstract] OR FBXW7[Title/Abstract] OR PALB2[Title/Abstract] OR CREBBP[Title/Abstract])) AND ("2007/01/01"[Date - Publication] : "2017/01/01"[Date - Publication])) AND hasabstract[text]'
all_pubmed_ids = pu.search_pubmed(search_term, retmax=500000)
print len(all_pubmed_ids)

248371


In [7]:
random_2000_pubmed_ids = random.sample(all_pubmed_ids, 2000)
print len(random_2000_pubmed_ids)

2000


In [8]:
with open(os.path.join(DESKTOP, "random_2000_pubmed_ids.txt"), "wb") as f:
    for _id in random_2000_pubmed_ids:
        f.write(_id + "\n")

In [6]:
# 1000_pubmed_abstracts_titles.tsv is the processed file created containing 1000 abstracts and titles from the 
# random_2000_pubmed_ids
df = pd.read_csv(os.path.join(base_dir, "1000_pubmed_abstracts_titles.tsv"), sep="\t", index_col=0, encoding='utf-8')

In [7]:
print df["abstract"][26703734]

The inside of a tumor often contains a hypoxic area caused by a limited supply of molecular oxygen due to aberrant vasculature. Hypoxia-inducible factors (HIFs) are major transcription factors that are required for cancer cells to adapt to such stress conditions. HIFs, complexed with the aryl hydrocarbon receptor nuclear translocator, bind to and activate target genes as enhancers of transcription. In addition to this common mechanism, the induction of the unfolded protein response and mTOR signaling in response to endoplasmic reticulum stress is also known to be involved in the adaptation to hypoxia conditions. Sp1 is a ubiquitously-expressed transcription factor that plays a vital role in the regulation of numerous genes required for normal cell function. In addition to the well-characterized stress response mechanisms described above, increasing experimental evidence suggests that Sp1 and HIFs collaborate to drive gene expression in cancer cells in response to hypoxia, thereby regul

In [8]:
new_df = pd.DataFrame(columns=["pmid", "gene", "abstract", "title", 
                               "gene in title", "gene in abstract", "gene freq in abstract", "gene in first two sent"])
count = 0
for (pmid, row) in df.iterrows():
    abstract = unicode(row["abstract"])
    title = unicode(row["title"])
    match_genes = genes_re.findall(title + ". " + abstract)
    match_genes = set([x.strip().lower() for x in match_genes])
    if match_genes:
        for gene in match_genes:
            temp_gene_re = re.compile('\s?' + gene.strip() + '\s?', re.IGNORECASE)
            matches = temp_gene_re.findall(abstract)
            sentences = nltk.sent_tokenize(abstract)
            new_df.loc[ count ] = [ pmid, gene, abstract, title, True if temp_gene_re.search(title) else False,
                                    True if len(matches) > 0 else False, len(matches), 
                                   True if temp_gene_re.search(". ".join(sentences[:2])) else False ]
            count += 1

new_df.set_index(keys=["pmid", "gene"], inplace=True)
new_df.to_excel(os.path.join(base_dir, "gene_importance_review.xlsx"))

In [4]:
pubmed_ids = []
regex = re.compile("\d+")
for ID in xls["pubmed_ID"]:
    match = regex.search(ID)
    if match:
        temp = [x for x in regex.findall(ID) if len(x) > 5]
        pubmed_ids.extend(temp)

In [5]:
print len(set(pubmed_ids))

399


In [8]:
article_generator = pu.get_medline(pubmed_ids)
article_dict = dict()
for article in article_generator:
    article_dict[article["PMID"]] = article

In [ ]:
pub_types = []
for article in article_dict.values():
    pub_type = tuple(sorted(article.get("PT")))
    if pub_type == ('Journal Article',):
        print article.get("PMID")
    pub_types.append(pub_type)
    
counter = Counter(pub_types)
print counter.most_common(20)

In [20]:
years = []
for article in article_dict.values():
    pub_date = datetime.datetime.strptime(article.get("DA"),"%Y%m%d")
    #print pub_date.year
    years.append(int(pub_date.year))

years_dist = nltk.FreqDist(years)
print years_dist.most_common(20)
print "Max is: %d" % max(years)
print "Min is: %d" % min(years)

[(2016, 67), (2015, 66), (2013, 56), (2014, 49), (2012, 47), (2011, 28), (2010, 21), (2009, 15), (2008, 13), (2006, 9), (2005, 7), (2004, 6), (2007, 6), (2017, 3), (1992, 1), (1998, 1), (2000, 1), (2001, 1), (2002, 1), (2003, 1)]
Max is: 2017
Min is: 1992


In [9]:
gene_pubmed_ids = set()
for gene_sym in set(xls["gene"]):
    IDS = list(xls[xls["gene"] == gene_sym]["pubmed_ID"])
    for ID in IDS:
        match = regex.search(ID)
        if match:
            temp = [x for x in regex.findall(ID) if len(x) > 5]
            gene_pubmed_ids.update([ (gene_sym, x) for x in temp ])
            
#print gene_pubmed_ids

In [14]:
count_gene_in_abstract, count_gene_in_first5, count_gene_in_first10, count_gene_in_title = (0, 0, 0, 0)
count_freq_1_if_not_first5 = 0
count_freq_gte2_first5_in_title = 0

for (gene_sym, pmid) in gene_pubmed_ids:
    article = article_dict[pmid]
    title = article.get("TI", "")
    abstract = article.get("AB", "")
    combined = title + "\n" + abstract
    sentences = nltk.sent_tokenize(abstract)
    count_gene_in_abstract += gene_sym in abstract
    count_gene_in_first5 += gene_sym in " ".join(sentences[:5])
    count_gene_in_first10 += gene_sym in " ".join(sentences[:10])
    count_gene_in_title += gene_sym in title
    frequency = len(re.findall(gene_sym, abstract))
    print "%s (%s)" % (pmid, gene_sym)
    print "%sGene in abstract: %s" % ("\t", gene_sym in abstract)    
    print "%sGene in first 5: %s" % ("\t", gene_sym in " ".join(sentences[:5]))
    print "%sGene in first 10: %s" % ("\t", gene_sym in " ".join(sentences[:10]))
    print "%sGene in title: %s" % ("\t", gene_sym in title)
    print "%sFrequency: %d" % ("\t", frequency)
    if (gene_sym in title) and not((gene_sym in " ".join(sentences[:5])) and (frequency > 1)):
        print "\t\tProblem"
    count_freq_1_if_not_first5 += (gene_sym not in " ".join(sentences[:5])) and (frequency < 2)
    count_freq_gte2_first5_in_title += (gene_sym in " ".join(sentences[:5])) and (frequency > 1) and (gene_sym in title)
    
print "Total gene, pubmed count: %d" % len(gene_pubmed_ids)
print "Total gene in abstract: %d" % count_gene_in_abstract
print "Total gene in first 5: %d" % count_gene_in_first5
print "Total gene in first 10: %d" % count_gene_in_first10
print "Total gene in title: %d" % count_gene_in_title
print "Total gene not in first 5 and freq < 2: %s" % count_freq_1_if_not_first5
print "Total gene freq > 1 and in first 5 and in title: %d" % (count_freq_gte2_first5_in_title)

23662903 (PIK3CA)
	Gene in abstract: False
	Gene in first 5: False
	Gene in first 10: False
	Gene in title: False
	Frequency: 0
22319199 (RET)
	Gene in abstract: False
	Gene in first 5: False
	Gene in first 10: False
	Gene in title: False
	Frequency: 0
26990290 (KRAS)
	Gene in abstract: True
	Gene in first 5: True
	Gene in first 10: True
	Gene in title: False
	Frequency: 2
15638953 (EGFR)
	Gene in abstract: True
	Gene in first 5: True
	Gene in first 10: True
	Gene in title: False
	Frequency: 2
23158522 (TSC1)
	Gene in abstract: False
	Gene in first 5: False
	Gene in first 10: False
	Gene in title: False
	Frequency: 0
23558169 (IDH1)
	Gene in abstract: True
	Gene in first 5: True
	Gene in first 10: True
	Gene in title: True
	Frequency: 10
26392102 (BRAF)
	Gene in abstract: True
	Gene in first 5: True
	Gene in first 10: True
	Gene in title: True
	Frequency: 5
23394218 (PIK3CA)
	Gene in abstract: False
	Gene in first 5: False
	Gene in first 10: False
	Gene in title: False
	Frequency: 0
19

	Gene in title: False
	Frequency: 1
26884591 (NTRK3)
	Gene in abstract: True
	Gene in first 5: True
	Gene in first 10: True
	Gene in title: True
	Frequency: 3
18594509 (PTEN)
	Gene in abstract: False
	Gene in first 5: False
	Gene in first 10: False
	Gene in title: False
	Frequency: 0
24960402 (ERBB2)
	Gene in abstract: False
	Gene in first 5: False
	Gene in first 10: False
	Gene in title: False
	Frequency: 0
24647572 (PTEN)
	Gene in abstract: True
	Gene in first 5: False
	Gene in first 10: True
	Gene in title: True
	Frequency: 2
		Problem
18235121 (KIT)
	Gene in abstract: True
	Gene in first 5: True
	Gene in first 10: True
	Gene in title: True
	Frequency: 1
		Problem
26469692 (PIK3CA)
	Gene in abstract: True
	Gene in first 5: False
	Gene in first 10: True
	Gene in title: False
	Frequency: 3
25239608 (KIT)
	Gene in abstract: True
	Gene in first 5: True
	Gene in first 10: True
	Gene in title: True
	Frequency: 7
23846731 (MAP2K1)
	Gene in abstract: False
	Gene in first 5: False
	Gene in f

26566875 (ARAF)
	Gene in abstract: True
	Gene in first 5: True
	Gene in first 10: True
	Gene in title: False
	Frequency: 4
27071922 (KRAS)
	Gene in abstract: True
	Gene in first 5: False
	Gene in first 10: True
	Gene in title: False
	Frequency: 1
23610105 (ERBB2)
	Gene in abstract: False
	Gene in first 5: False
	Gene in first 10: False
	Gene in title: False
	Frequency: 0
22134241 (CHEK2)
	Gene in abstract: False
	Gene in first 5: False
	Gene in first 10: False
	Gene in title: False
	Frequency: 0
22294718 (PIK3CA)
	Gene in abstract: True
	Gene in first 5: True
	Gene in first 10: True
	Gene in title: False
	Frequency: 2
26565381 (NTRK2)
	Gene in abstract: False
	Gene in first 5: False
	Gene in first 10: False
	Gene in title: False
	Frequency: 0
23099652 (EGFR)
	Gene in abstract: False
	Gene in first 5: False
	Gene in first 10: False
	Gene in title: False
	Frequency: 0
21690569 (KRAS)
	Gene in abstract: False
	Gene in first 5: False
	Gene in first 10: False
	Gene in title: False
	Frequenc

In [10]:
fgfr3_xls = pd.read_excel(os.path.join(base_dir, "LOE_Output_FGFR3_DB06589_CEreview_5-10-17.xlsx"), dtype={'PMID' : 'str'})

In [11]:
pmids = list(fgfr3_xls["PMID"])
articles = list(pu.get_medline(pmids))

In [12]:
publishers = map(lambda x: " ".join(x) if isinstance(x, list) else x, [ article.get("CI") for article in articles ])

In [12]:
print publishers

[None, '(c) 2011 The Authors. British Journal of Pharmacology (c) 2011 The British Pharmacological Society.', None, None, None, None, 'Copyright (c) 2015 The Authors. Published by Elsevier Inc. All rights reserved.', None, None, None, None, None, None, None, None, None, None, None, None, None, None, '(c) 2015 S. Karger AG, Basel.', None, None, None, None, 'Copyright (c) 2015 Elsevier Ltd. All rights reserved.', 'Copyright (c) 2015 Elsevier B.V. All rights reserved.', None, None, None, None, None, None, None, None, None, None, 'Published by Elsevier Inc.', None, None, None, None, None, None, None, 'Copyright (c) 2017 Elsevier B.V. All rights reserved.', None, 'Copyright (c) 2015 Elsevier Inc. All rights reserved.', '(c) 2015 The Authors. British Journal of Pharmacology published by John Wiley & Sons Ltd on behalf of The British Pharmacological Society.', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '(c)2015 American Association for Canc

In [14]:
wiley = [ publisher for publisher in publishers if (publisher is not None) ]
print len(wiley)

19


In [ ]:
review_types = set(["Review", "Systematic Reviews"])
for article in articles:
    pt = article.get("PT")
    abstract = article.get("AB")
    if len(set(pt).intersection(review_types)) > 0:
        print "%s: %s" % (article.get("PMID"), ("review" in abstract) or ("Review" in abstract))
    #if ("review" in abstract) or ("Review" in abstract):
    #    print "%s: %s" % (article.get("PMID"), len(set(pt).intersection(review_types)) > 0)

In [13]:
count = 0
for article in articles:
    abstract = article.get("AB", "")
    sentences = nltk.sent_tokenize(abstract)
    count += ("FGFR3" in ". ".join(sentences[:5]))
    
print "Count: %d" % count
print len(articles)

Count: 8
101
